# Projet, f = 1. : Partie 1/2

Considérons le problème de elliptique suivant : nous cherchons une fonction $u:[0,1]\to\mathbb{R}$ qui vérifie le problème
\begin{equation}
\left\lbrace
\begin{aligned}
-aL * u1''(x) = f(x),&&0<x<x0,\\
-aR * u2''(x) = f(x),&&x0<x<1 \\
&u(0)=u(1)=0,  \\
u(x0) = Eps = 0 && noté && E && pour && l'instant
\end{aligned}
\right.
\end{equation}
où $f:[0,1]\to\mathbb{R}$ est une fonction régulière (par exemple continue).

Ce problème peut se résoudre directement. La solution est donnée par la formule
\begin{equation}
u1(x) = x \int_0^E \int_0^t f(s) \, \operatorname{d}\!s \, \operatorname{d}\!t - \int_0^x \int_0^t f(s) \, \operatorname{d}\!s \, \operatorname{d}\!t . 
\end{equation}

\begin{equation}
u2(x) = x \int_E^1 \int_0^t f(s) \, \operatorname{d}\!s \, \operatorname{d}\!t - \int_0^x \int_0^t f(s) \, \operatorname{d}\!s \, \operatorname{d}\!t . 
\end{equation}



In [1]:
#include <iostream>
#include <vector>
#include <stdexcept>

#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"

#include "mesh1D.hpp"
#include "solution1D.hpp"
#include "matrix.hpp"
#include "lu.hpp"

###  1.  

Créez un objet de type `parameters` contenant les informations suivantes
* `xmin` l'abscisse du bord gauche ($0$ dans notre cas)
* `xmax` l'abscisse du bord droit ($1$ dans notre cas)
* `Nx` le nombre de points du maillage (à votre convenance mais ne soyez pas trop gourmand dans un premier temps)
* `node-centered` qui doit prendre la valeur $1$ de manière à construire un maillage centré sur les noeuds (et pas sur les cellules)

###  2.  

Créez une `lambda`-fonction pour le second membre de l'équation. Ici nous pourrons commencer avec la fonction $f(x) = 1$, $\forall x\in[0,1]$.

In [2]:
double max(std::vector<double> x)
{
    double maximum =x[0];
    for(int i = 1; i<x.size();i++)
    {
        if(x[i]>maximum)
        {maximum = x[i];}
    }
    
    return maximum;
}



In [3]:
double max_ab(double a, double b)
{
    if(a>b)
    {return a;}
    else {return b;}
}

In [4]:
double aL = (rand()/(double)RAND_MAX ) * (5-0) + 0; // Nmbre aléa entre 0 et 5
double aR = (rand()/(double)RAND_MAX ) * (1.5-0) + 0 ;
double x0 = (rand()/(double)RAND_MAX )*(0.7-0.3) + 0.3;
double ld = (x0-1)/(2*(aL-aR-(aL/x0)))

In [5]:
std::cout<< ld << std::endl ;

0.0497208


In [6]:
parameters p = {
    {"xmin",0.},
    {"xmax",x0},
    {"Nx",32},
    {"cell-centered",1}
};

parameters p2 = {
    {"xmin",x0},
    {"xmax",1.},
    {"Nx",32},
    {"cell-centered",1}
};


In [7]:
double f(double x ) {return 1.;}

###  3.  

Dans le cas où $f(x)=1$, la solution évidente du problème s'écrit
$$ u(x) = \frac{1}{2} x(1-x).$$

Créez une fonction qui prend en argument $x$ un vecteur de `double` et qui retourne $y$ un vecteur de même taille tel que 
$$ y_k = u(x_k) .$$

On prend f = 1, d'où :
------------
------------

\begin{equation}
u1(x) = -\frac{x²}{2aL} + x(\frac{λ}{x0}+\frac{x0}{2aL})  
\end{equation}

--------------------------------
-------------------------------


\begin{equation}
u2(x) = -\frac{x²}{2aR} + x(\frac{λ}{x0-1} - \frac{x0 + 1}{2aR}) - \frac{x0}{2aR} - \frac{λ}{x0 -1}
\end{equation}

---------------------
---------------------

In [8]:
// Solution pour u1
std::vector<double> sol(std::vector<double> x)  
{
    std::vector<double> y(x.size());
    for(int k = 0; k < x.size(); k++){
        y[k] = -(x[k]*x[k])*(0.5/aL) + x[k]*(ld/x0 + 0.5*x0/aL);
    }
    return y;
}



In [9]:
// Solution pour u2
std::vector<double> sol2(std::vector<double> x)  
{
    std::vector<double> y(x.size());
    for(int k = 0; k < x.size(); k++){
        y[k] = -(x[k]*x[k])*(0.5/aR) + x[k]*(ld/(x0 -1) + 0.5*(x0+1)/aR) - 0.5*x0/aR - ld/(x0-1);
    }
    return y;
}

###  4.  

* Fabriquez tous les éléments du problème : 
    * les solution `S1` et `S1` (objet de type `solution1D`) à partir des paramètres,
    * la matrice $A$ du Laplacien dont la taille est $N{\times}N$ avec $N=N_x-2$ où $N_x$ est le nombre de points du maillage comprenant les deux points de bord, 
    * le second membre $F$ qui doit être un `vector` de taille $N$ également ;
* résolvez le système linéaire $Au=F$ en utilisant la décomposition $LU$ de la matrice $A$. Le vecteur `u_in` obtenu est un vecteur de taille $N$ ;
* copiez les valeurs de `u_in` à leur place dans le vecteur `S.u` qui est de taille $N_x=N+2$.

In [10]:
solution1D S(p);
auto N = S.m.Nx - 2 ;
auto Nx = S.m.Nx;
auto invh = (N+1)/x0;
matrix A = 2 * identity(N,N) - identity(N,N,1) - identity(N,N,-1);

matrix A1 = aL * invh*invh* A;

std::vector<double> F(N);
std::transform(std::next(S.m.x.begin()),std::prev(S.m.x.end()),F.begin(),f);
F[N-1] += aL*ld*invh*invh;

matrix LU1 = lu(A1);

std::vector<double> sol_n = solve_lu(LU1,F);
for(int k = 1 ; k<S.u.size() - 1;++k)
    S.u[k] = sol_n[k-1];

S.u[Nx-1] = ld;


In [11]:
solution1D S2(p2);


std::vector<double> F2(N);

auto invh2 = (N+1)/(1-x0);
std::transform(std::next(S2.m.x.begin()),std::prev(S2.m.x.end()),F2.begin(),f);
F2[0] += aR*ld*invh2*invh2;

matrix A2 = A*aR*invh2*invh2;
matrix LU2 = lu(A2);
std::vector<double> sol2_n = solve_lu(LU2,F2);
for(int k = 1 ; k<S.u.size() - 1;++k)
    S2.u[k] = sol2_n[k-1];
S2.u[0] = ld;

###  5.  

Affichez le résultat obtenu et comparez le avec la solution exacte.
Essayez d'obtenir un graphique semblable à celui-ci.

<img src="img/TP02_Laplacien.png", width='75%'>

In [12]:
double maxi1 = max(S.u);
double maxi2 = max(S2.u);
double maximax = max_ab(max(S.u),max(S2.u));

In [13]:
xpl::linear_scale sx, sy, sx2,sy2;
sx.min = p["xmin"], sx.max = p2["xmax"];
sx2.min = p["xmin"], sx2.max = p2["xmax"];
sy.min = 0., sy.max = maximax;

// fine mesh for the exact solution
auto NN = 1025;
double dx = x0/(NN-1);
std::vector<double> xx(NN);
for (auto i=0; i<NN; ++i)
    xx[i] = i*dx;

double dx2 = (1-x0)/(NN-1);
std::vector<double> xx2(NN);
for (auto i=0; i<NN; ++i)
    xx2[i] = x0+i*dx2;

auto ax_x = xpl::axis_generator(sx)
    .label("x")
    .finalize();
auto ax_y = xpl::axis_generator(sy)
    .orientation("vertical")
    .side("left")
    .finalize();


auto line = xpl::lines_generator(sx, sy)
    .x(xx)
    .y(sol(xx))
    .colors(std::vector<std::string>({"red"}))
    .labels(std::vector<std::string>({"exact"}))
    .display_legend(true)
    .finalize();

auto line2 = xpl::lines_generator(sx2, sy)
    .x(xx2)
    .y(sol2(xx2))
    .colors(std::vector<std::string>({"orange"}))
    .labels(std::vector<std::string>({"exact 2"}))
    .display_legend(true)
    .finalize();

auto scatter = xpl::scatter_generator(sx, sy)
    .x(S.m.x)
    .y(S.u)
    .colors(std::vector<xtl::xoptional<std::string>>{"navy"})
    .labels(std::vector<std::string>({"numeric"}))
    .display_legend(true)
    .marker("circle")
    .finalize();

auto scatter2 = xpl::scatter_generator(sx2, sy)
    .x(S2.m.x)
    .y(S2.u)
    .colors(std::vector<xtl::xoptional<std::string>>{"green"})
    .labels(std::vector<std::string>({"numeric 2"}))
    .display_legend(true)
    .marker("circle")
    .finalize();

auto fig = xpl::figure_generator()
    .padding_x(0.025)
    .padding_y(0.1)
    .title("solution with N = " + std::to_string(S.m.Nx))
    .legend_location("top-left")
    .finalize();

auto fig2 = xpl::figure_generator()
    .padding_x(0.025)
    .padding_y(0.1)
    .title("solution with N = " + std::to_string(S2.m.Nx))
    .legend_location("top-left")
    .finalize();



fig.add_mark(line);
fig.add_mark(line2);
fig.add_mark(scatter);
fig.add_mark(scatter2);
fig.add_axis(ax_x);
fig.add_axis(ax_y);
fig.display();

// ---------------------- //


A Jupyter widget

In [14]:
std::cout<< "x0 = " << x0 << std::endl;
std::cout<< "aL = " << aL << std::endl;
std::cout<< "aR = " << aR << std::endl;

x0 = 0.453401
aL = 3.39648
aR = 1.40204
